# Parameter sweeps

The `DifferentialEquations.jl` and `JuliaDynamics` ecosystems are useful
for making highly optimized parameter sweeps of dynamical systems.

In particular, we'll look at:
- `OrdinaryDiffEq` (DifferentialEquations)
- `ChaosTools` (JuliaDynamics)

The tools from JuliaDynamics' `DynamicalSystems.jl` (of which `ChaosTools.jl` is
a part) are documented in tutorial format very well in the book by Datseris &
Parlitz.

In [1]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()

  Activating project at `~/Dev/siam-student-conf-2025/notebooks`


# Lyapunov spectrum analysis

We'll compute two sweeps:
- *Leading Lyapunov exponent*, which shows parameter regions corresponding to
  chaotic dynamics.
- *Lyapunov dimension*, which shows the fractal dimension of the attractor in the
  state space.

Let's compute these for a single point in parameter space to see how they work.

In [2]:
using OrdinaryDiffEq

In [3]:
function lorenz!(du, u, p, t)
  # Extract parameters.
  ρ, σ, β = p

  # Extract state variables.
  x, y, z = u
  
  # Define the Lorenz system equations.
  du[1] = σ * (y - x)
  du[2] = x * (ρ - z) - y
  du[3] = x * y - β * z

  # Return the derivatives; we will need this for BifurcationKit.jl.
  du
end

lorenz! (generic function with 1 method)

In [4]:
p = [
  28.0,   # ρ
  10.0,   # σ
  8.0/3.0 # β
]

u0 = [
  10.0, # x
  10.0, # y
  10.0  # z
]

tspan = (0.0, 1e5)

(0.0, 100000.0)

In [5]:
# Integrate.
prob = ODEProblem(lorenz!, u0, tspan, p)
solution = solve(prob, Tsit5(), abstol=1e-6, reltol=1e-6, maxiters=1e8)

retcode: Success
Interpolation: specialized 4th order "free" interpolation
t: 4873280-element Vector{Float64}:
      0.0
      0.009514760366390658
      0.016430864584616352
      0.02650558552705793
      0.036483269891708386
      0.048304910472091636
      0.06091963672639215
      0.07500378592597223
      0.0897335013317697
      0.10429140733157334
      ⋮
  99999.83155722987
  99999.85154178432
  99999.87194480235
  99999.8929670181
  99999.9148431344
  99999.93788476517
  99999.96254809712
  99999.98948254695
 100000.0
u: 4873280-element Vector{Vector{Float64}}:
 [10.0, 10.0, 10.0]
 [10.073350228413272, 11.57867143976095, 10.766654654858943]
 [10.211390698013433, 12.680686851172009, 11.41292581784568]
 [10.523697450488259, 14.220591934758016, 12.496745092939497]
 [10.945147506897248, 15.664808984114842, 13.748435183262233]
 [11.562974012007302, 17.25153870135197, 15.478876348655254]
 [12.330104644216737, 18.749756750581213, 17.63791198675786]
 [13.268273684112305, 20.094114147

In [6]:
# Plot solution.

using GLMakie

# Extract the solution components.
xs = [u[1] for u in solution.u]
ys = [u[2] for u in solution.u]
zs = [u[3] for u in solution.u]

# Create the 3D plot.
fig = Figure()
ax = Axis3(
  fig[1, 1], 
  xlabel = "x", 
  ylabel = "y", 
  zlabel = "z",
  title = "Lorenz system trajectory"
)

# Plot the solution.
lines!(ax, xs, ys, zs, linewidth = 1)

# Display the figure.
fig

ChaosTools.jl wants the system specified not as an `ODEProblem`, but instead as
a `DynamicalSystem`, which is a type defined by ChaosTools.jl.
The appropriate subtype of `DynamicalSystem` for the Lorenz system is
`CoupledODEs`.

In [7]:
using ChaosTools

# Set up the system.
lorenz_system = CoupledODEs(
  lorenz!,
  u0,
  p
)

3-dimensional CoupledODEs
 deterministic: true
 discrete time: false
 in-place:      true
 dynamic rule:  lorenz!
 ODE solver:    Tsit5
 ODE kwargs:    (abstol = 1.0e-6, reltol = 1.0e-6)
 parameters:    [28.0, 10.0, 2.6666666666666665]
 time:          0.0
 state:         [10.0, 10.0, 10.0]


In [8]:
# Compute the Lyapunov spectrum.
λ = lyapunovspectrum(
  lorenz_system, # The DynamicalSystem.
  Int(3e2);      # Integration steps.
  Ttr = 1e2,     # Transient time.
  Δt = 1e-1      # Time between orthonormalization steps.
)

3-element Vector{Float64}:
   0.8976573932058167
   0.006348284187339487
 -14.570599449397237

In [11]:
# Calculate the Lyapunov dimension (Kaplan-Yorke dimension).
# This is defined as k + sum(λs[1:k])/|λs[k+1]|, where k is the largest
# integer such that the sum of the first k Lyapunov exponents is non-negative.
function lyapunov_dimension(λ)
    k = 0
    sum_λ = 0.0
    
    # Find the largest k where the sum is still non-negative.
    for (i, λ) in enumerate(λ)
        sum_λ += λ
        if sum_λ < 0
            break
        end
        k = i
    end
    
    # If all exponents are positive or k is the last index, return k.
    if k == length(λ) || k == 0
        return k
    end
    
    # Calculate the Kaplan-Yorke dimension.
    return k + sum(λ[1:k])/abs(λ[k+1])
end

lyapunov_dimension (generic function with 1 method)

In [12]:
lyap_dim = lyapunov_dimension(λ)
println("Estimated Lyapunov dimension: $lyap_dim")

Estimated Lyapunov dimension: 2.0620431355986906


We should expect a Lyapunov dimension of $2.06 \pm 0.01$.

# Sweeps over $\rho, \sigma$

In [13]:
using StaticArrays

In [135]:
# Define parameter range.
# ρs = range(10.0, 150.0, length=30);
ρs = range(0.0, 110.0, length=64);
# σs = range(0.0, 80.0, length=30);
σs = range(20.0, 55.0, length=64);

In [136]:
# Parameter values from the Cartesian product ρs × σs × {β}.
params = [[ρ, σ, p[3]] for ρ in ρs for σ in σs]

4096-element Vector{Vector{Float64}}:
 [0.0, 20.0, 2.6666666666666665]
 [0.0, 20.555555555555557, 2.6666666666666665]
 [0.0, 21.11111111111111, 2.6666666666666665]
 [0.0, 21.666666666666668, 2.6666666666666665]
 [0.0, 22.22222222222222, 2.6666666666666665]
 [0.0, 22.77777777777778, 2.6666666666666665]
 [0.0, 23.333333333333332, 2.6666666666666665]
 [0.0, 23.88888888888889, 2.6666666666666665]
 [0.0, 24.444444444444443, 2.6666666666666665]
 [0.0, 25.0, 2.6666666666666665]
 ⋮
 [110.0, 50.55555555555556, 2.6666666666666665]
 [110.0, 51.111111111111114, 2.6666666666666665]
 [110.0, 51.666666666666664, 2.6666666666666665]
 [110.0, 52.22222222222222, 2.6666666666666665]
 [110.0, 52.77777777777778, 2.6666666666666665]
 [110.0, 53.333333333333336, 2.6666666666666665]
 [110.0, 53.888888888888886, 2.6666666666666665]
 [110.0, 54.44444444444444, 2.6666666666666665]
 [110.0, 55.0, 2.6666666666666665]

In [137]:
function lorenz_static(u, p, _)
  # Extract parameters.
  ρ, σ, β = p

  # Extract state variables.
  x, y, z = u
  
  # Define the Lorenz system equations.
  du = SVector{3}(
    σ * (y - x),
    x * (ρ - z) - y,
    x * y - β * z
  )

  # Return the derivatives.
  du
end

lorenz_static (generic function with 1 method)

In [138]:
function lorenz_static_jac(u, p, _)
  # Extract parameters.
  ρ, σ, β = p

  # Extract state variables.
  x, y, z = u

  # Define the Lorenz system Jacobian.
  jac = SMatrix{3, 3}(
     -σ,  σ,  0,
    ρ-z, -1, -x,
      y,  x, -β
  )

  # Return the Jacobian.
  jac
end

lorenz_static_jac (generic function with 1 method)

In [139]:
# Calculate Lyapunov spectra in parallel threads.

ds = CoupledODEs(lorenz_static, u0, p)
tands = TangentDynamicalSystem(ds; J = lorenz_static_jac)

# Preallocate the array for storing the Lyapunov spectra.
λs = zeros(length(params), 3)

# Since `DynamicalSystem`s are mutable, we need to copy to parallelize.
systems = [deepcopy(tands) for _ in 1:Threads.nthreads()-1]
pushfirst!(systems, tands)

Threads.@threads for i in eachindex(params)
    system = systems[Threads.threadid()]
    set_parameters!(system, params[i])
    λs[i, :] .= lyapunovspectrum(system, Int(1e3); Ttr = 1e2)
end

# Display the calculated Lyapunov spectra.
λs

4096×3 Matrix{Float64}:
  1.2701e-9    -1.0       -2.66665
  3.8232e-10   -1.0       -2.66666
  6.00584e-11  -1.0       -2.66666
 -1.04356e-9   -1.0       -2.66666
  5.03125e-10  -1.0       -2.66666
  7.63052e-10  -1.0       -2.66666
 -5.00384e-10  -1.0       -2.66666
 -1.42866e-9   -1.0       -2.66666
  1.1759e-9    -1.0       -2.66666
  2.75876e-10  -1.0       -2.66666
  ⋮                      
  1.13217      -1.2813   -Inf
  1.19899      -1.5526   -Inf
  1.25837      -1.84196  -Inf
  1.31334      -2.15578  -Inf
  1.36505      -2.50327  -Inf
  1.41004      -2.90697  -Inf
  1.45293      -3.39728  -34.7822
  1.49037      -4.06062  -34.6433
  1.52255      -5.16614  -34.4582

In [140]:
# Store Leading Lyapunov exponents.
LLEs = [λs[i,1] for i in 1:size(λs)[1]]

# Store Lyapunov dimensions.
LDs = Float64[]
for i in 1:size(λs)[1]
  try
    push!(LDs, lyapunov_dimension(λs[i,:]))
  catch
    push!(LDs, NaN)
  end
end

In [141]:
# Plot both LLEs and LDs side by side in one figure.
fig = Figure(size = (900, 400))

# LLEs plot.
ax1 = Axis(
  fig[1, 1];
  xlabel = L"\rho",
  ylabel = L"\sigma",
  title = "Leading Lyapunov Exponent"
)
hm1 = heatmap!(
    ax1,
    ρs,
    σs,
    reshape(LLEs, (length(ρs), length(σs))),
    colormap = :thermal
)
Colorbar(fig[1, 2], hm1, label = L"\lambda_1")

# LDs plot.
ax2 = Axis(
  fig[1, 3];
  xlabel = L"\rho",
  ylabel = L"\sigma",
  title = "Lyapunov Dimension"
)
hm2 = heatmap!(
    ax2,
    ρs,
    σs,
    reshape(LDs, (length(ρs), length(σs))),
    colormap = :thermal,
    # Clip LDs below 2.
    colorrange = (max(
      1.95,
      minimum(filter(!isnan, LDs))),
      maximum(filter(!isnan, LDs))
    )
)
Colorbar(fig[1, 4], hm2, label = L"D_L")

fig